<a href="https://colab.research.google.com/github/Young4424/Machine-Learning-Deep-learning-Myself/blob/main/00_ML_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import numpy as np
import pandas as pd

In [146]:
data = pd.read_csv('titanic.csv')

In [147]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [148]:
# 결측값 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [149]:
data = data.drop(columns = 'Cabin')

In [150]:
# data['Age'].fillna(data['Age'].median())

In [151]:
from sklearn.impute import SimpleImputer

In [152]:
# imputer = SimpleImputer(strategy = 'constant', fill_value = 0)
# imputer.fit_transform(data[['Age']])

In [153]:
imputer = SimpleImputer(strategy = 'mean')
data['Age'] = imputer.fit_transform(data[['Age']])

In [154]:
data['Age'] = data['Age'].astype(int)

In [155]:
# data.columns

In [156]:
drop_cols = ['PassengerId', 'Name', 'Ticket']
numeric_cols = ['Age', 'SibSp', 'Parch', 'Fare']
categorical_cols = ['Pclass', 'Sex', 'Embarked']

In [157]:
data = data.drop(columns = drop_cols)

In [158]:
num_feats = data[numeric_cols]
cat_feats = data[categorical_cols]

In [159]:
# 연속형 변수들의 처리
    # 1. 결측값, 이상치를 처리 - 대체(imputation)
    # 2. 스케일링

imputer = SimpleImputer(strategy = 'median')
num_feats = pd.DataFrame(data = imputer.fit_transform(num_feats), columns = numeric_cols)

In [160]:
from sklearn.preprocessing import MinMaxScaler

In [161]:
scaler = MinMaxScaler()
num_feats = pd.DataFrame(data = scaler.fit_transform(num_feats), columns = numeric_cols)

In [162]:
# num_feats.head()

In [163]:
# 범주형 변수의 처리
    # 1. 결측치 처리 - 대체(Imputation), 최빈값
    # 2. 인코딩 - One Hot encoding

imputer = SimpleImputer(strategy = 'most_frequent')
cat_feats = pd.DataFrame(data = imputer.fit_transform(cat_feats), columns = categorical_cols)

In [164]:
cat_feats = pd.get_dummies(cat_feats)

<ipython-input-164-f55e48d8fb31>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  cat_feats = pd.get_dummies(cat_feats)


In [165]:
features = pd.concat([num_feats, cat_feats], axis = 1)

In [166]:
# (data["Name"].str.split().str.get(1)).value_counts()

In [167]:
# data['Married'] = data["Name"].str.split().str.get(1)

In [168]:
# data['Married'] = data['Married'].apply(lambda x: x if x in ['Mr.', 'Mrs.', 'Miss.', 'Master.'] else 'others')

In [169]:
# pd.get_dummies(data[['Married']])

In [170]:
# features.to_csv('titanic_feats.csv', index = False)

In [171]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [172]:
targets = data['Survived']

In [173]:
X_train, X_valid, y_train, y_valid = train_test_split(features, targets, test_size = 0.2)

In [174]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [175]:
from sklearn.metrics import classification_report

In [176]:
y_pred = model.predict(X_valid)

In [177]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.75      0.81       123
           1       0.59      0.79      0.67        56

    accuracy                           0.76       179
   macro avg       0.74      0.77      0.74       179
weighted avg       0.79      0.76      0.77       179



In [178]:
scores = cross_val_score(
    estimator = DecisionTreeClassifier(),
    X = features, y = targets,
    cv = 5
)

In [179]:
scores.mean()

0.7778105580315109

In [180]:
for i in range(3, 10):

    scores = cross_val_score(
            estimator = DecisionTreeClassifier(max_depth = i, random_state = 89),
            X = features, y = targets,
            cv = 5
        )

    print(f"max_dpeth = {i}일때, {scores.mean()}")

max_dpeth = 3일때, 0.8103132257862029
max_dpeth = 4일때, 0.805881614462369
max_dpeth = 5일때, 0.8114493754315486
max_dpeth = 6일때, 0.8148327160881301
max_dpeth = 7일때, 0.8137028435126483
max_dpeth = 8일때, 0.8125980792166217
max_dpeth = 9일때, 0.8159437574540205


In [181]:
from sklearn.neighbors import KNeighborsClassifier

In [182]:
scores = cross_val_score(
    estimator = KNeighborsClassifier(n_neighbors = 5),
    X = features, y = targets,
    cv = 5
)

scores.mean()

0.7957567007720796

In [183]:
scores = cross_val_score(
    estimator = KNeighborsClassifier(n_neighbors = 10),
    X = features, y = targets,
    cv = 5
)

scores.mean()

0.7957567007720797

In [184]:
for i in range(3, 15):

    scores = cross_val_score(
                estimator = KNeighborsClassifier(n_neighbors = i),
                X = features, y = targets,
                cv = 5
                )

    print(f"K = {i}일때, {scores.mean()}")

K = 3일때, 0.7946393823363254
K = 4일때, 0.7901512773837173
K = 5일때, 0.7957567007720796
K = 6일때, 0.7946017199171427
K = 7일때, 0.7946142740568702
K = 8일때, 0.7990961019396146
K = 9일때, 0.7991023790094783
K = 10일때, 0.7957567007720797
K = 11일때, 0.7968740192078337
K = 12일때, 0.8092210156299041
K = 13일때, 0.8024794425961961
K = 14일때, 0.8002385286548239


In [185]:
from sklearn.svm import SVC

In [186]:
scores = cross_val_score(
            estimator = SVC(kernel = 'rbf'),
            X = features, y = targets,
            cv = 5
            )

scores.mean()

0.8114556525014125

In [187]:
scores = cross_val_score(
            estimator = SVC(kernel = 'linear'),
            X = features, y = targets,
            cv = 5
            )

scores.mean()

0.7867365513778168

In [188]:
scores = cross_val_score(
            estimator = SVC(kernel = 'poly'),
            X = features, y = targets,
            cv = 5
            )

scores.mean()

0.8114556525014125

# Parameter Tuning

In [189]:
from sklearn.model_selection import RandomizedSearchCV

In [190]:
from sklearn.utils.extmath import randomized_range_finder
model = DecisionTreeClassifier()
param_grid = {'max_depth' : [3,5,7],
              'min_samples_split' : [5,10,15]}

rand_search = RandomizedSearchCV(model,param_grid, n_iter = 5, scoring = 'roc_auc', cv = 5)
rand_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=5,
                   param_distributions={'max_depth': [3, 5, 7],
                                        'min_samples_split': [5, 10, 15]},
                   scoring='roc_auc')

In [191]:
rand_search.best_params_

{'min_samples_split': 10, 'max_depth': 3}

In [192]:
from sklearn.neighbors import KNeighborsClassifier

In [193]:
model = KNeighborsClassifier()
param_grid = {'weights' : ['uniform','distance'],
              'n_neighbors' : range(5,10),
              'algorithm':['ball_tree','kd_tree','brute']}

rand_search = RandomizedSearchCV(model,param_grid, n_iter = 5, scoring = 'roc_auc', cv = 5)
rand_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'n_neighbors': range(5, 10),
                                        'weights': ['uniform', 'distance']},
                   scoring='roc_auc')

In [194]:
rand_search.best_score_

0.8298692258737793

In [195]:
rand_search.best_params_

{'weights': 'uniform', 'n_neighbors': 8, 'algorithm': 'brute'}

# 반복문을 이용한 파라미터 튜닝

In [196]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [197]:
clfs = [
   (
        KNeighborsClassifier(),              # 사용하려는 모델
        {'n_neighbors': [3,5,7,9,11],        # 최적화하려는 하이퍼파라미터
         'weights': ['uniform','distance']}
    ),
    (
        DecisionTreeClassifier(random_state=0),
        {'max_depth':[3,5,7],
         'min_samples_split':[2,3,4]}
    ),
    (
        SVC(),
        {'kernel': ['linear', 'poly', 'rbf'],
         'degree': [3,4,5],
         'C':[1.0, 1.2, 1.4]}
    ),
    (
        XGBClassifier(),
            {'max_depth': [3, 4, 5, 6, 7, 8],
            'sampling_method': ['uniform', 'subsample', 'gradient_based']}
    ),
    (
        LGBMClassifier(),
        {'max_depth': [3, 4, 5, 6, 7, 8],
        'learning_rate ': [0.1, 0.05, 0.01]}
    )
]

In [198]:
import time
import tqdm # 어디 즈음에 돌아가는지 보여주는 바를 나타냄
from sklearn.metrics import roc_auc_score

In [199]:
clfs_tuned = []  # 튜닝된 모델을 저장
for clf, param_grid in tqdm.tqdm_notebook(clfs):
    start = time.time()
    rand_search = RandomizedSearchCV(clf, param_grid, n_iter=10, scoring='roc_auc',
                                     cv=3, random_state=0, n_jobs=-1)

    rand_search.fit(X_train, y_train)
    clf_name = type(clf).__name__
    y_pred = rand_search.predict(X_valid)
    clf_score = roc_auc_score(y_valid, y_pred)
    print('{:30s} {:30f} {:.1f}'.format(clf_name, clf_score, time.time() - start))
    clfs_tuned.append((clf_name, rand_search, clf_score))

<ipython-input-199-26e62f985f4c>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for clf, param_grid in tqdm.tqdm_notebook(clfs):


  0%|          | 0/5 [00:00<?, ?it/s]

KNeighborsClassifier                                 0.809161 0.3


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


DecisionTreeClassifier                               0.791304 0.2
SVC                                                  0.783246 0.4


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
12 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1490, in fit
    self._Booster = train(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 620, in inner_f
  

XGBClassifier                                        0.788037 2.0
[LightGBM] [Warning] learning_rate is set=0.1, learning_rate= will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] Unknown parameter: 0.05
LGBMClassifier                                       0.804297 0.8


In [200]:
 clfs_tuned

[('KNeighborsClassifier',
  RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
                     param_distributions={'n_neighbors': [3, 5, 7, 9, 11],
                                          'weights': ['uniform', 'distance']},
                     random_state=0, scoring='roc_auc'),
  0.8091608594657376),
 ('DecisionTreeClassifier',
  RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=0),
                     n_jobs=-1,
                     param_distributions={'max_depth': [3, 5, 7],
                                          'min_samples_split': [2, 3, 4]},
                     random_state=0, scoring='roc_auc'),
  0.7913037166085947),
 ('SVC',
  RandomizedSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
                     param_distributions={'C': [1.0, 1.2, 1.4],
                                          'degree': [3, 4, 5],
                                          'kernel': ['linear', 'poly', 'rbf']},
                     random_state=0, sc

# 앙상블

## voting

In [201]:
from sklearn.ensemble import VotingClassifier, StackingClassifier, BaggingClassifier

In [202]:
knn_tuned = clfs_tuned[0][1]
dt_tuned = clfs_tuned[1][1]
svc_tuned = clfs_tuned[2][1]

In [203]:
ensemble_model = VotingClassifier( estimators= [('DT', dt_tuned),('KNN',knn_tuned),('SVC',svc_tuned)],voting = 'hard')

In [204]:
ensemble_model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


VotingClassifier(estimators=[('DT',
                              RandomizedSearchCV(cv=3,
                                                 estimator=DecisionTreeClassifier(random_state=0),
                                                 n_jobs=-1,
                                                 param_distributions={'max_depth': [3,
                                                                                    5,
                                                                                    7],
                                                                      'min_samples_split': [2,
                                                                                            3,
                                                                                            4]},
                                                 random_state=0,
                                                 scoring='roc_auc')),
                             ('KNN',
                              RandomizedSearchCV(cv=3,
                                                 estimator=KNeighborsClassifier(),
                                                 n_jobs=-1,
                                                 param_distributions={'n_neighbors': [3,
                                                                                      5,
                                                                                      7,
                                                                                      9,
                                                                                      11],
                                                                      'weights': ['uniform',
                                                                                  'distance']},
                                                 random_state=0,
                                                 scoring='roc_auc')),
                             ('SVC',
                              RandomizedSearchCV(cv=3, estimator=SVC(),
                                                 n_jobs=-1,
                                                 param_distributions={'C': [1.0,
                                                                            1.2,
                                                                            1.4],
                                                                      'degree': [3,
                                                                                 4,
                                                                                 5],
                                                                      'kernel': ['linear',
                                                                                 'poly',
                                                                                 'rbf']},
                                                 random_state=0,
                                                 scoring='roc_auc'))])

In [205]:
y_pred =  ensemble_model.predict(X_valid)
roc_auc_score(y_valid,y_pred)

0.8116289198606272

In [206]:
clfs_tuned

[('KNeighborsClassifier',
  RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
                     param_distributions={'n_neighbors': [3, 5, 7, 9, 11],
                                          'weights': ['uniform', 'distance']},
                     random_state=0, scoring='roc_auc'),
  0.8091608594657376),
 ('DecisionTreeClassifier',
  RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=0),
                     n_jobs=-1,
                     param_distributions={'max_depth': [3, 5, 7],
                                          'min_samples_split': [2, 3, 4]},
                     random_state=0, scoring='roc_auc'),
  0.7913037166085947),
 ('SVC',
  RandomizedSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
                     param_distributions={'C': [1.0, 1.2, 1.4],
                                          'degree': [3, 4, 5],
                                          'kernel': ['linear', 'poly', 'rbf']},
                     random_state=0, sc

# Bagging

In [208]:
ensemble_model = BaggingClassifier(base_estimator = dt_tuned, n_estimators = 10)

In [209]:
ensemble_model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space

BaggingClassifier(base_estimator=RandomizedSearchCV(cv=3,
                                                    estimator=DecisionTreeClassifier(random_state=0),
                                                    n_jobs=-1,
                                                    param_distributions={'max_depth': [3,
                                                                                       5,
                                                                                       7],
                                                                         'min_samples_split': [2,
                                                                                               3,
                                                                                               4]},
                                                    random_state=0,
                                                    scoring='roc_auc'))

In [210]:
y_pred =  ensemble_model.predict(X_valid)
roc_auc_score(y_valid,y_pred)

0.7872386759581882

# Stacking

In [211]:
ensemble_model = StackingClassifier(estimators=[('DT', dt_tuned),('KNN',knn_tuned),('SVC',svc_tuned)],
                   final_estimator= XGBClassifier())

In [215]:
ensemble_model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/

StackingClassifier(estimators=[('DT',
                                RandomizedSearchCV(cv=3,
                                                   estimator=DecisionTreeClassifier(random_state=0),
                                                   n_jobs=-1,
                                                   param_distributions={'max_depth': [3,
                                                                                      5,
                                                                                      7],
                                                                        'min_samples_split': [2,
                                                                                              3,
                                                                                              4]},
                                                   random_state=0,
                                                   scoring='roc_auc')),
                               ('KNN',
                                RandomizedSearchCV(cv=3,
                                                   estimator=KNeighborsClassifier(),
                                                   n_jobs=-1,
                                                   param_distributions={'n_neighbors': [3,
                                                                                        5,
                                                                                        7,
                                                                                        9,
                                                                                        11],
                                                                        'we...
                                                 gpu_id=None, grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=100, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 predictor=None,
                                                 random_state=None, ...))

In [216]:
y_pred =  ensemble_model.predict(X_valid)
roc_auc_score(y_valid,y_pred)

0.7677119628339142